In [20]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import numpy as np

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Open Multilingual Wordnet

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [22]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
63,"Intended as light entertainment, this film is ...",negative
130,This should have been a movie about Sam and hi...,negative
727,What is supposed to be a simple generic myster...,negative
404,The one who says that Lucio Fulci is not one o...,positive
803,"As a South African, living in South Africa aga...",negative


In [23]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [24]:
df = normalize_text(df)
df.head()

,review,sentiment
63,intended light entertainment film indeed succe...,negative
130,movie sam wife glorious peter falk equally glo...,negative
727,supposed simple generic mystery plot involving...,negative
404,one say lucio fulci one important name history...,positive
803,south african living south africa year stay uk...,negative


In [25]:
df['sentiment'].value_counts()

sentiment
positive    251
negative    249
Name: count, dtype: int64

In [26]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [27]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
63,intended light entertainment film indeed succe...,0
130,movie sam wife glorious peter falk equally glo...,0
727,supposed simple generic mystery plot involving...,0
404,one say lucio fulci one important name history...,1
803,south african living south africa year stay uk...,0


In [28]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [29]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [33]:
import os
from dotenv import load_dotenv
import mlflow
import dagshub

# Load environment variables
load_dotenv()

True

In [34]:
from dagshub.auth import add_oauth_token

# This will open a browser window for authentication
add_oauth_token()

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=78c71572-4518-4437-afc7-c58829c3d3d3&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=234dec1602109dc2741b203610ae0dd83d756a0dcabdce05a7845e53a8a8ddf4




In [19]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/jaggusuperhit/capstone.mlflow')
dagshub.init(repo_owner='jaggusuperhit', repo_name='capstone', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

Accessing as jaggusuperhit

Initialized MLflow to track repo "jaggusuperhit/capstone"

Repository jaggusuperhit/capstone initialized!

2025/04/27 20:17:33 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/556dc414262244f9815f80266bed4298', creation_time=1745765254055, experiment_id='0', last_update_time=1745765254055, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [35]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-04-27 20:32:38,186 - INFO - Starting MLflow run...
2025-04-27 20:32:40,113 - WARNING - Retrying (JitteredRetry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /jaggusuperhit/capstone.mlflow/api/2.0/mlflow/runs/create
2025-04-27 20:32:40,704 - INFO - Logging preprocessing parameters...
2025-04-27 20:32:42,680 - INFO - Initializing Logistic Regression model...
2025-04-27 20:32:42,682 - INFO - Fitting the model...
2025-04-27 20:32:42,779 - INFO - Model training complete.
2025-04-27 20:32:42,783 - INFO - Logging model parameters...
2025-04-27 20:32:43,216 - INFO - Making predictions...
2025-04-27 20:32:43,217 - INFO - Calculating evaluation metrics...
2025-04-27 20:32:43,245 - INFO - Logging evaluation metrics...
2025-04-27 20:32:45,653 - INFO - Saving and logging the model...
2025/04/27 20:33:00 WARNING 

🏃 View run aged-ant-630 at: https://dagshub.com/jaggusuperhit/capstone.mlflow/#/experiments/0/runs/30baf7b7ae224b8cb75a5748c5a3f514
🧪 View experiment at: https://dagshub.com/jaggusuperhit/capstone.mlflow/#/experiments/0
